# Equal-Weight S&P 500 Index Fund

In [1]:
import numpy as np 
import pandas as pd
import requests 
import xlsxwriter 
import math 

New libraries:

- requests: used for HTTP requests.

- xlswriter : used to save Excel files from a Python script

### Importing List of Stocks

S&P 500 Index actually changes with time, but in this notebook we will consider a static version of it, which you can find in the folder under the name _sp_500_stocks.csv_. As you wil see, it is a file with the tickers/symbols of each company in S&P 500.



In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## API Token

In this project, we'll we working with the IEX Cloud API token, which will be our data provider. API Tokens should be stored that doesn't get pushed to the Git repository, as it contains sensitive information. This file is the one called `secrets.py`

The API we'll be using generates random data which mimics real-world data. The main reason why we are using this API is because it is free.

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Making calls in APIs is tricky, as the method is different for each of them. In this case, we would like to extract the following information:

* Market capitalization for each stock
* Price of each stock

This is done by using the code below. Documentation related to the API can be found [here ](https://iexcloud.io/docs/api/) (look for _Testing Sandbox_ and _Quote_):

In [5]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}' #token needed at
#the end to verify that we can have access to that information
data = requests.get(api_url).json() 
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'GMN)S DRCA TNAE(AEOSLSG LQB/ELTKA',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'foilfaic',
 'close': None,
 'closeTime': None,
 'closeSource': 'liafcifo',
 'high': None,
 'highTime': 1651631525970,
 'highSource': 'ee  I miEtrcerlpiXa',
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 125.64,
 'latestSource': 'IEX real time price',
 'latestTime': '10:30:34 AM',
 'latestUpdate': 1626594298107,
 'latestVolume': None,
 'iexRealtimePrice': 128.64,
 'iexRealtimeSize': 33,
 'iexLastUpdated': 1620631681125,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 127.12,
 'previousVolume': 118221691,
 'change': 1.41,
 'changePercent': 0.011,
 'volume': None,
 'iexMarketPercent': 0.0117716612705

As you can see, this provides lots of information, but we only want the market capitalization and the latest price of the stock:

In [6]:
price = data["latestPrice"]
market_cap = data["marketCap"]


## Adding Stocks Data to a DataFrame

The title is self-explanatory:

In [7]:
my_columns = ["Ticker","Stock Price",
              "Market Capitalization","Number of Shares to Buy"]

final_dataframe = pd.DataFrame(columns=my_columns)

final_dataframe = final_dataframe.append(pd.Series([symbol,price,market_cap,'N/A'], index = my_columns),
                       ignore_index=True) #in order to append it must be given as pd.Series

final_dataframe



,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,125.64,2102277756368,N/A


## Looping Through Every Ticker in the Stock List

The code below works, but it is _very_ slow, as HTTP requests are extremely slow. You can try the code with the first 10 tickers. Fortunately, there are some methods that improve the performance of the calls:

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"][:10]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}' #token needed at
    data = requests.get(api_url).json() 
    price = data["latestPrice"]
    market_cap = data["marketCap"]
    final_dataframe = final_dataframe.append(pd.Series([stock,price,market_cap,'N/A'], index = my_columns),
                       ignore_index=True) 

        

In [9]:
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,123.930,38463577040,N/A
1,AAL,18.470,10940183518,N/A
2,AAP,161.793,10611199127,N/A
3,AAPL,129.220,2175003731873,N/A
4,ABBV,110.590,189270673594,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code. IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def groups(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n] #yield returns a generator

In [11]:
symbol_groups = list(groups(stocks["Ticker"],100))
symbol_strings = []

for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i])) #we are joining each list and separating them by a comma, basically
    
symbol_strings
    

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

And now we perform the batch (look for _Batch Requests_ in API documentation):

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
     batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
     data = requests.get(batch_api_call_url).json()
     for symbol in symbol_string.split(','): #opposite of join:
        final_dataframe = final_dataframe.append(
            pd.Series(
             [symbol,
              data[symbol]['quote']['latestPrice'], #for some reason it is inside something called quote
              data[symbol]['quote']['marketCap'],
               "N/A"],
            index=my_columns),
            ignore_index=True)
        

In [13]:
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.950,37870141707,N/A
1,AAL,18.341,11227960814,N/A
2,AAP,155.864,10978583460,N/A
3,AAPL,124.500,2148740344840,N/A
4,ABBV,110.640,197319592619,N/A


Since we are making 5 calls instead of 500 (we have 5 groups of 100 tickers), this piece of code is _much_ faster.

## Calculating the Number of Shares To Buy

First of all, we enter with the keyboard the value of our portfolio:

In [14]:
portfolio_size = input("Enter the value of your portfolio")

try:
    val = float(portfolio_size)
except ValueError:
    print("Please, enter a numerical value")
    portfolio_size = input("Enter the value of  your portfolio")
    val=float(portfolio_size)

Enter the value of your portfolio 1000


And now we calculate the size of the positions for each ticker. Once we have done this, we can update the row of the DataFrame corresponding to the number of shares to buy, as this is just the price of the stocks divided by its size in our portfolio:

In [15]:
position_size = val/len(final_dataframe.index) #since we want EW portfolio, the size of each position is the same

final_dataframe["Number of Shares to Buy"] = final_dataframe["Stock Price"]/position_size



**BEAR IN MID THAT WE HAVE FRACTIONAL SHARES**

## Formatting An Excel Output

For this matter, we will use the package XlsxWriter. First of all, we have to initialize the XlsxWriter object:


In [17]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_dataframe.to_excel(writer,'Recommended Trades',index=False)

Now we have to create the formats we need for the .xlsx file:

* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [18]:
background_color = '#0a0a23'
font_color = "#ffffff"


string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {   
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

Now, we must apply these formats to the columns of our .xlsx file:

In [20]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    } #types of formats

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}' #set format to this column, (e.g. A:A)
                                                   , 20 #width of column
                                                   , column_formats[column][1] #set this format
                                                  )
    writer.sheets['Recommended Trades'].write(f'{column}1' #write in this position
                                              , column_formats[column][0] #write this thing
                                              , string_format #write it in this format 
                                             )

Finally, we save the .xlsx file, which should appear in the folder in which this notebook is located:

In [21]:
writer.save()

If you open the file, you should find a black sheet with white font called "Recommended Trades".

## References

Algorithmic Trading Using Python - Full Course : https://www.youtube.com/watch?v=xfzGZB4HhEE&t=14236s
